# Fermetures (*closures*), fonctions anonymes capturant leurs contextes


Les fermetures (*closures*) en Rust sont des fonctions anonymes qui peuvent être sauvegardées dans des variables et passées comme arguments à d'autres fonctions.
Vous pouvez les créer à un endroit et les utiliser (évaluer) dans un autre contexte.

La syntaxe et les capacité des fermetures font qu'elles sont très faciles à utiliser dans beaucoup de contexte.
Appeler une fermeture se fait de la même façon que l'apple à une fonction.
Cependant, le typage des paramètres et du type de retour peuvent être inférés par le compilateur.

Quelques caractéristiques des fermetures :

* Les paramètres sont déclarés entre `|  |`
* Le corp est une expression (rappel: un block `{}` est une expression)
* Elles peuvent capturer les variables de leur environnement


In [ ]:
fn main() {
    // Incrément en utiliser des fonctions et des fermetures
    fn  function (i: i32) -> i32 { i + 1 }

    // les fermetures sont anonymes
    // ici nous les assignons à des variables pour les utiliser
    let closure_annotated = |i: i32| -> i32 { i + 1 }; // avec annotations de type
    let closure_inferred  = |i     |          i + 1  ; // avec inférence de type

    let i = 1;
    println!("function: {}", function(i));
    println!("closure_annotated: {}", closure_annotated(i));
    println!("closure_inferred: {}", closure_inferred(i));
}
main();

Les fermetures peuvent capturer leur environnement, contrairement aux fonctions.

In [2]:
fn main() {
    let value: i32 = 3;

    let f = |x| x + value; // <-- la fermeture fait référence à value

    println!("{}", f(2));
}
main();

5


Par défaut, les fermetures capturent leur environnement comme des références (immutables).

In [ ]:
fn main() {
    let value = String::from("Hello !");
    let tell = || { println!("{}", value); };
    tell();
    println!("value is still here: {}", value);
}
main();

Les fermetures peuvent également capturer des références mutables.
Dans ce cas, la variable qui contient fermeture doit être déclarée mutable.

In [ ]:
fn main() {
    let mut value = 0;
    {
        let mut inc = || { value += 1; };
        inc();
    }
    println!("value = {}", value);
}
main();

Le *borrow-checker* s'assure de la sûreté des référence mutables, même pour les fermetures.

Parfois il peut être utile que la fermeture prenne la propriété (*ownership*) de leur environnement.
Pour faire cela, il convient d'utiliser le mot clé `move` devant la définition de la fermeture.
La propriété de toutes les variables référencées dans la fermeture sera transféré à la fermeture.

In [6]:
fn main() {
    let a = String::from("Hello ");
    let b = String::from("World !");

    let tell = move || { print!("{}", a); }; // here a is moved to the closure

    tell();
    
    tell();
    println!("{}", b); // this is OK
    // println!("{}", a); // this is an error
}
main();

Hello Hello World !


Les fermetures permettent de tirer parti de l'inférence de type et de ne pas spécifier explicitement le typage des paramètres et du type de retour.
Le compilateur va bien sûr inférer un type concret.

Capture de l'environnement
---

Les fermetures peuvent capturer des valeurs de leur environnement de trois façons, qui correspondent exactement aux trois façons dont une valeur peut être passée en paramètre : 1) en prenant la propriété, 2) en empruntant de manière mutable, et 3) en empruntant de manière non-mutable.
Ces 3 façons s'encode sous la forme des 3 traits `Fn` :

* `FnOnce` consomme les variables capturées de l'environnement (*closure's environment*). Pour que ces variables capturées puissent être consommées, la fermeture doit pendre la propriété de ces variables et les déménager (*move*) dans le contexte de la fermeture. Le mot `Once` dans le nom représente le fait que cette fermeture ne peut être exécutée qu'une seule fois, comme son exécution consomme les variables capturées.
* `FnMut` peut changer son environnement en empruntant de manière mutable les variables de son contexte.
* `Fn` emprunte son environnement de manière non-mutable.

# Exercices

**Exercice 1** Compléter le code suivant

In [16]:
fn adder(value1: i32,value2: i32) -> /*impl Fn(i32) ->*/ i32 {
    let total= |x , y| x+y;
    return total(value1,value2);
}

fn test_adder() {
    assert_eq!(adder(3,2), 5);
    // it's equal to
    //let closure = addr(3);
    //closure(2);
    
    assert_eq!(adder(4,6), 10);
    
        println!("{}", adder(3,2));
}
test_adder();

5


In [17]:
fn closure_user<F: Fn(i32)->i32>(closure: F) {
    assert_eq!(closure(3), 4);
    assert_eq!(closure(5), 6);
    assert_eq!(closure(7), 8);
}

fn test_closure_user() {
    let mut call_count = 0;
    closure_user(|x| {
        call_count += 1;
        x + 1 
    });
    assert_eq!(call_count, 3);
}
test_closure_user();

Error: the precise format of `Fn`-family traits' type parameters is subject to change

Error: missing generics for trait `FnMut`

Error: the value of the associated type `Output` in `FnOnce` must be specified

Error: trait objects must include the `dyn` keyword

**Solution**

In [ ]:
fn adder(value: i32) -> impl Fn(i32) -> i32 {
    move |x| x + value
}

fn test_adder() {
    assert_eq!(adder(3)(2), 5);
    assert_eq!(adder(4)(6), 10);
}
test_adder();


fn closure_user<F: FnMut(i32)->i32>(mut closure: F) {
    assert_eq!(closure(3), 4);
    assert_eq!(closure(5), 6);
    assert_eq!(closure(7), 8);
}

fn test_closure_user() {
    let mut call_count = 0;
    closure_user(|x| {
        call_count += 1;
        x + 1 
    });
    assert_eq!(call_count, 3);
}
test_closure_user();